In [5]:
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import rnn  # rnn stuff temporarily in contrib, moving back to code in TF 1.1
import os
import time
import math
import numpy as np
import my_txtutils as txt
import sys

tf.set_random_seed(0)
tf.reset_default_graph()

SEQLEN = 30
BATCHSIZE = 20
ALPHABETSIZE = txt.ALPHASIZE
INTERNALSIZE = 128
NLAYERS = 3


keep_prob = 0.8    # some dropout


# Load Shakespeare Data
shakedir = "shakespeare/*.txt"
codetext, valitext, bookranges = txt.read_data_files(shakedir, validation=True)

# display some stats on the data
epoch_size = len(codetext) // (BATCHSIZE * SEQLEN)
txt.print_data_stats(len(codetext), len(valitext), epoch_size)

# DEFINE PLACEHOLDERS
keep_prob_placeholder = tf.placeholder(tf.float32, name='keep_prob')  # dropout parameter
batchsize = tf.placeholder(tf.int32, name='batchsize')


# inputs
X = tf.placeholder(tf.uint8, [None, None], name='X')    # [ BATCHSIZE, SEQLEN ]
Xo = tf.one_hot(X, ALPHABETSIZE, 1.0, 0.0)                 # [ BATCHSIZE, SEQLEN, ALPHASIZE ]

# expected outputs = same sequence shifted by 1 since we are trying to predict the next character
Y_ = tf.placeholder(tf.uint8, [None, None], name='Y_')  # [ BATCHSIZE, SEQLEN ]
Yo_ = tf.one_hot(Y_, ALPHABETSIZE, 1.0, 0.0)               # [ BATCHSIZE, SEQLEN, ALPHASIZE ]

# input state
Hin = tf.placeholder(tf.float32, [None, INTERNALSIZE*NLAYERS], name='Hin')  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]

# using NLAYERS layers of GRU cells, unrolled SEQLEN=30 times
# dynamic_rnn infers SEQLEN from the size of the inputs Xo

cells = [rnn.GRUCell(INTERNALSIZE) for _ in range(NLAYERS)]

# "naive dropout" implementation
dropcells = [rnn.DropoutWrapper(cell,input_keep_prob=keep_prob_placeholder) for cell in cells]
multicell = rnn.MultiRNNCell(dropcells, state_is_tuple=False)
multicell = rnn.DropoutWrapper(multicell, output_keep_prob=keep_prob_placeholder)  # dropout for the softmax layer

Yr, H = tf.nn.dynamic_rnn(multicell, Xo, dtype=tf.float32, initial_state=Hin)
# Yr: [ BATCHSIZE, SEQLEN, INTERNALSIZE ]
# H:  [ BATCHSIZE, INTERNALSIZE*NLAYERS ] # this is the last state in the sequence


# Softmax layer implementation:
# Flatten the first two dimension of the output [ BATCHSIZE, SEQLEN, ALPHASIZE ] => [ BATCHSIZE x SEQLEN, ALPHASIZE ]
# then apply softmax layer. This way, the weights and biases are shared across unrolled time steps.

W = tf.Variable(tf.random_normal([INTERNALSIZE, ALPHABETSIZE]))
B = tf.Variable(tf.random_normal([ALPHABETSIZE]))

Yflat = tf.reshape(Yr, [-1, INTERNALSIZE])    # [ BATCHSIZE x SEQLEN, INTERNALSIZE ]
Ylogits = tf.matmul(Yflat, W) + B     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]

Yflat_ = tf.reshape(Yo_, [-1, ALPHABETSIZE])     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]

loss = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Yflat_)  # [ BATCHSIZE x SEQLEN ]
loss = tf.reshape(loss, [batchsize, -1])      # [ BATCHSIZE, SEQLEN ]

Yo = tf.nn.softmax(Ylogits, name='Yo')        # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Y = tf.argmax(Yo, 1)                          # [ BATCHSIZE x SEQLEN ]
Y = tf.reshape(Y, [batchsize, -1], name="Y")  # [ BATCHSIZE, SEQLEN ]

train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

# stats for display
seqloss = tf.reduce_mean(loss, 1)
batchloss = tf.reduce_mean(seqloss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(Y_, tf.cast(Y, tf.uint8)), tf.float32))

# for display: init the progress bar
DISPLAY_FREQ = 50
_50_BATCHES = DISPLAY_FREQ * BATCHSIZE * SEQLEN
progress = txt.Progress(DISPLAY_FREQ, size=111+2, msg="Training on next "+str(DISPLAY_FREQ)+" batches")

# init
istate = np.zeros([BATCHSIZE, INTERNALSIZE*NLAYERS])  # initial zero input state
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
step = 0

# training loop
for x, y_, epoch in txt.rnn_minibatch_sequencer(codetext, BATCHSIZE, SEQLEN, nb_epochs=2):

    # train on one minibatch
    feed_dict = {X: x, Y_: y_, Hin: istate, keep_prob_placeholder: keep_prob, batchsize: BATCHSIZE}
    _, y, ostate = sess.run([train_step, Y, H], feed_dict=feed_dict)

    # display a short text generated with the current weights and biases (every 150 batches)
    if step // 3 % _50_BATCHES == 0:
        print()
        print("Generating some random text...")
        
        ry = np.array([[txt.convert_from_alphabet(ord("K"))]])
        rh = np.zeros([1, INTERNALSIZE * NLAYERS])

        for k in range(1000):
            ryo, rh = sess.run([Yo, H], feed_dict={X: ry, keep_prob_placeholder: 1.0, Hin: rh, batchsize: 1})
            rc = txt.sample_from_probabilities(ryo, topn=10 if epoch <= 1 else 2)
            sys.stdout.write(chr(txt.convert_to_alphabet(rc)))
            ry = np.array([[rc]])

        print()
        print("FINISHED GENERATING RANDOM TEXT")
        print()

    # display progress bar
    progress.step(reset=step % _50_BATCHES == 0)

    # loop state around
    istate = ostate
    step += BATCHSIZE * SEQLEN
    
print()
print("-----FINISHED TRAINING-----")


Loading file shakespeare/merchantofvenice.txt
Loading file shakespeare/loverscomplaint.txt
Loading file shakespeare/othello.txt
Loading file shakespeare/hamlet.txt
Loading file shakespeare/kingjohn.txt
Loading file shakespeare/various.txt
Loading file shakespeare/sonnets.txt
Loading file shakespeare/asyoulikeit.txt
Loading file shakespeare/titusandronicus.txt
Loading file shakespeare/2kinghenryvi.txt
Training text size is 0.9178218841552734MB with 149.2587890625KB set aside for validation. There will be 1604 batches per epoch

Generating some random text...
$G'ddq[^.$'R[G.$qqq '  qGO$  G^q$$RR .R <G.y OR OqqRG .q $G.R GG$y$^   '$y<q$.R$ $.^R'<.^^q<B.R.^B^^ q^^ ^OB$^^qq$B^qy#<O +qO ^++$$^qqO < ^ O^^#. # R^O R^^^RRR^^ R^OR BZqB$qO^Rqq ^q<OB + $q+^RqRRG<^O+<O<GO<.^ ^ .G dd  q.^d[^dRy.'yh^^y y^ ^OOB$#BB+qOR< +dqq +$G .<G. R. R qqG'MR RR$<<O^M M$yq^..Oq.yR$.$  .R^^GBq^$h  $RBy^^R$ $q BqGR$  R<Gyy^$^ B^^qqBqBR +BB#^^RO,O^O++^$OR

O^+q RO$  ^ R.O$dG.f$.. G  .d.$dRR^GRR.G.y^  y  yq Rq < yqO <q

	an fhot har won sout ant nowinge nde mes, bod sa teralad maceninedite ous a meader thean, thivh shet, wirinow it shenderin nos aneseranes malasdt

IOARONONT	I i an he lee toreradas oftom ther snoton thirim mend ale than theee, ank

HAMRRENI	were noundesit eule chith tinteramim of ro seore thy us
Torethil  aelave wimom ood nins her hin ther
	Ond nteram hans misee, what nowetest oun bn huar sontofe nof is hareer

	Whers th to teas to mate 
FINISHED GENERATING RANDOM TEXT


0%                                        Training on next 50 batches                                        100%
0%                                        Training on next 50 batches                                        100%
0%                                        Training on next 50 batches                                        100%
Generating some random text...
dqu asesot de te thar atis ta ty s as  a ceroted nowras hase sh a her sissalo bomy
	Thicth, and efaad you heng chound steer sanerdethe anging s eftome

 hARR	No, srat,	And ing,s the asener in ber wher all sto thes his atrus see oure hour thet in me these
	I wichars shavu hore ie as thes and are are on mo asels
	And tat e sperese chateessalators,
	To eestelad all onen in musd bent sanien ome the eftarlare, hive more my tore o ssangor,
Lof i hrid ter hing tomen bean nes a bees, thor my sins losss;
	Hor tas seose toul hass a derind tyas.

AEnTE
	The sorle a soud trets, meshoutteshir of hount ty suide the trend, whe sareed hat the e ong, dound mes,
	In thos ade tosessatriog try or halss mers andine.
	Ore sore soon aricor ou te t nere tir til so lank,
	Ther wilg hout be he his mord ound nowradnd
 I ald word and wy lowess, wae tee the erich of tare ond
	In mond no re ter nath ne tore il her hat tondes
	I whar astere senst oredstored sured nigr hack;	And andyofertrerind astae,
	The eamae nes and benge his and hich: I held

TIMATII	Wo as, the sape ein is:	Theu wlut, ard ble to teee,
	It ow he doss earet not mant e efreres amy hirsse anleress


0%                                        Training on next 50 batches                                        100%
0%                                        Training on next 50 batches                                        100%
Generating some random text...
G', thy affad ard

	His to he thimk ent thete samoun. he mon sanow
	Betrter wathen soued hear hess
	Thoulinit, a lirt ong tay the tet the bentile in.
	The fir the sestore with it spell come in hine; hald; I hosh in borats so me here feelthan tir cave the the pit beact.
	I to live in spure inss the and are, whe wath, in, at how sun a dsunce.
	Whe enels and ani in at trly woth m you most; mer in wnth, hit, bore

RTIASDONALIUN	                                                                                                            his wall fou alid in aseded
	Te priokso wet thin that oun the and

	Inther with aroshe thongry saligh lave; I pale a my. theue
	And arowet him hat aral ardeathe thighteach as inghintto mast
 The firte,
	I 

In my lond widh a ang fore hone firnend?
	Tam shill so ther sherss they, in stlowese, a darinttela
	I drll ang the his thace have, and that core yout her foth as arany;
Ang hem orame tuman hom ane a cooes ont yot what to thee hom o the hen,
	What weacs.

THUROThack hime issone, I hall theurselall:
	But nothaty tha keed.

ORANIL	If hom many hand he firnore and him
FINISHED GENERATING RANDOM TEXT


0%                                        Training on next 50 batches                                        100%
0%                                        Training on next 50 batches                                        100%
0%                                        Training on next 50 batches                                        100%
Generating some random text...
, hhoso, swend str is with bight stavis.
	And beatt a st bug fielition
Whe paven shosleear wo flond har sume we to harted the bad d dights heals;
Thomewer ow oriof, I moreshin. Hhar the hees not.

	And not wise ans the ame tond